In [94]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [95]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0
    
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

def clean_column(df,column):
    """
    Функция для очистки от колонки от пустых значений
    """
    df[column] = df[column].astype(str) # Приводим к строковому виду
    df[column] = df[column].apply(lambda x:x.replace('nan','Отсутствует'))
    df[column] = df[column].apply(lambda x:x.strip()) # очищаем от пробельных символов(на случай если в ячейке стоит просто пробел или несколько пробелов)
    df[column] = df[column].apply(lambda x:x if x else 'Отсутствует') # Если пустая строка то заменяем на значение Не заполнено

def prepare_column_to_int(cell):
    """
    Функция для обработки ячеек в инт, чтобы они в экселе потом не отображались как строки
    """
    try:
        float_value = float(cell)
        int_value = int(float_value)
        return int_value
    except ValueError:
        return cell

In [96]:
region = 0

file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Заголовок Таблица 3 от 08.11.xlsx'

In [97]:
# Создаем список колонок которые нужно загрузить
use_cols_upp = list(range(11))
use_cols_table3 = list(range(34))

# Загружаем датафреймы
# При загрузке делаем строковыми колонки квартал,выдел,год лесоустройства, чтобы потом не получить неприятность при мердже
df = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols_upp,dtype={4:str,5:str,11:str})# Реестр УПП 
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=use_cols_table3,dtype={4:str,5:str,15:str}) # Таблица 3

In [98]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [99]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\n'

In [100]:
df.head()

,1,2,3,4,5,6,7,8,9,10,11
0,Ангоянское,Ангоянское,,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",2005
1,Ангоянское,Ангоянское,NaN,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",0
2,Ангоянское,Ангоянское,NaN,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",0
3,Ангоянское,Ангоянское,NaN,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",0
4,Ангоянское,Ангоянское,NaN,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",0


In [101]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га','11':'Год лесоустройства'},inplace=True)



In [102]:

clean_column(df,'Урочище') # очищаем от пустых строк и пробелов

In [103]:
df.head(5)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",7,8,9,10,Год лесоустройства
0,Ангоянское,Ангоянское,Отсутствует,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",2005
1,Ангоянское,Ангоянское,Отсутствует,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",0
2,Ангоянское,Ангоянское,Отсутствует,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",0
3,Ангоянское,Ангоянское,Отсутствует,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",0
4,Ангоянское,Ангоянское,Отсутствует,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",0


In [104]:
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: x.replace(',', '.'))

# Группируем
df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                       'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                               })

df_upp=df_upp.astype(str)


In [105]:
# Извлекаем индексы в колонки
df_upp = df_upp.reset_index()

In [106]:
df_upp.head()


,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га"
0,Ангоянское,Ангоянское,Отсутствует,100,10,259.0
1,Ангоянское,Ангоянское,Отсутствует,100,11,144.0
2,Ангоянское,Ангоянское,Отсутствует,100,12,78.0
3,Ангоянское,Ангоянское,Отсутствует,100,13,127.0
4,Ангоянское,Ангоянское,Отсутствует,100,14,22.0


In [107]:
# делаем строковыми первые 5 колонок в датафрейме Реестр УПП
df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']] = df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']].astype(str)

In [108]:
# Добавляем разделители для колонок квартала и выдела
df_upp['Номер лесного квартала'] = df_upp['Номер лесного квартала'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
df_upp['Номер лесотаксационного выдела'] = df_upp['Номер лесотаксационного выдела'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел

In [109]:
# Создаем колонку с идентификатором
df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела'] ].sum(axis=1)
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))

In [110]:
# Обрабатываем колонки таблицы 3
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [111]:
# Очищаем от нан и пустых строк
clean_column(df_table_3,'7')

In [112]:
# делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
df_table_3[['5','6','7','8','9']] = df_table_3[['5','6','7','8','9']].astype(str)
# Добавляем разделители для колонок квартала и выдела
df_table_3['8'] = df_table_3['8'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
df_table_3['9'] = df_table_3['9'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел

In [113]:
# Создаем колонку с идентификатором
df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9'] ].sum(axis=1)
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

In [114]:
# Мержим по полю айди
merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)

In [115]:
# Заменяем площадь в таблице 3 в случае если  такой же выдел есть в реестре
merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])

In [116]:
# Приводим к строковому виду чтобы очистить от разделителей кв. и в.
merged_df['8'] = merged_df['8'].astype(str)
merged_df['9'] = merged_df['9'].astype(str)
# Очищаем от разделителей

merged_df['8'] = merged_df['8'].apply(lambda x:x.replace('кв.',''))
merged_df['9'] = merged_df['9'].apply(lambda x:x.replace('в.',''))

In [117]:
# Приводим к инту колонки чтобы отсортировать их

merged_df['8'] = merged_df['8'].apply(prepare_column_to_int)
merged_df['9'] = merged_df['9'].apply(prepare_column_to_int)

In [118]:
merged_df.sort_values(['5','6','7','8','9'],inplace=True)

# Присваиваем колонке 10 значение колонки созданной колонки 10
#merged_df['10'] = merged_df['Площадь']

In [119]:
merged_df.to_excel('Тест.xlsx',index=False)

In [120]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Нет в таблице 3'
wb.create_sheet(title='Нет в реестре УПП', index=1)
wb.create_sheet(title='Выделы с обновленной площадью', index=2)

<Worksheet "Выделы с обновленной площадью">

In [121]:
# Записываем каждый датафрейм в соответсвующий лист
# Записи реестра которых нет в таблице 3
left_df = merged_df[merged_df['_merge'] == 'left_only']
left_df.drop(left_df.iloc[:,7:],axis=1,inplace=True)

In [122]:
# # Очищаем от пробелов и пустых строк 
# left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
# left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.strip())

In [123]:
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Нет в таблице 3'].append(r)
wb['Нет в таблице 3'].column_dimensions['A'].width = 30
wb['Нет в таблице 3'].column_dimensions['B'].width = 30
    

In [124]:
# Записываем каждый датафрейм в соответсвующий лист
# Записи которые есть только в таблице 3
right_df = merged_df[merged_df['_merge'] == 'right_only']
right_df.drop(right_df.iloc[:,:6],axis=1,inplace=True)

In [125]:
# Удаляем лишние колонки
right_df.drop(right_df.iloc[:,10:],axis=1,inplace=True)

right_df.drop(columns=['1','2','3','4'],axis=1,inplace=True)

In [126]:
right_df

,ID,5,6,7,8,9
138837,БабушкинскоеБабушкинскоеОтсутствуеткв.238в.12,Бабушкинское,Бабушкинское,Отсутствует,238,12
138838,БабушкинскоеБабушкинскоеОтсутствуеткв.238в.12,Бабушкинское,Бабушкинское,Отсутствует,238,12
138839,БабушкинскоеБабушкинскоеОтсутствуеткв.253в.17,Бабушкинское,Бабушкинское,Отсутствует,253,17
138840,БабушкинскоеБабушкинскоеОтсутствуеткв.263в.3,Бабушкинское,Бабушкинское,Отсутствует,263,3
138841,БабушкинскоеБабушкинскоеОтсутствуеткв.264в.7,Бабушкинское,Бабушкинское,Отсутствует,264,7
...,...,...,...,...,...,...
138832,ХандагатайскоеЧелутаевскоеОтсутствуеткв.161в.14,Хандагатайское,Челутаевское,Отсутствует,161,14
138833,ХандагатайскоеЧелутаевскоеОтсутствуеткв.161в.17,Хандагатайское,Челутаевское,Отсутствует,161,17
138834,ХандагатайскоеЧелутаевскоеОтсутствуеткв.173в.11,Хандагатайское,Челутаевское,Отсутствует,173,11
138835,ХандагатайскоеЧелутаевскоеОтсутствуеткв.173в.15,Хандагатайское,Челутаевское,Отсутствует,173,15


In [127]:
# # Очищаем от пробелов и строки названия урочища не заполнено
# right_df['7'] = right_df['7'].apply(lambda x:x.replace('Название урочища не заполнено',''))
# right_df['7'] = right_df['7'].apply(lambda x:x.strip())

In [128]:
# Переименовываем колонки
right_df.columns = ['ID','Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']

In [129]:
# Записываем на лист
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Нет в реестре УПП'].append(r)

wb['Нет в реестре УПП'].column_dimensions['A'].width = 30
wb['Нет в реестре УПП'].column_dimensions['B'].width = 30
wb['Нет в реестре УПП'].column_dimensions['C'].width = 30

In [130]:
# Записываем каждый датафрейм в соответсвующий лист
both_df = merged_df[merged_df['_merge'] == 'both']

In [131]:
both_df.drop(both_df.iloc[:,17:-1],axis=1,inplace=True)

In [132]:
both_df.head(5)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,4,5,6,7,8,9,10,Площадь
69,Ангоянское,Ангоянское,Отсутствует,кв.32,в.1,13.0,АнгоянскоеАнгоянскоеОтсутствуеткв.32в.1,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,13.0
70,Ангоянское,Ангоянское,Отсутствует,кв.32,в.1,13.0,АнгоянскоеАнгоянскоеОтсутствуеткв.32в.1,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,300.0,13.0
71,Ангоянское,Ангоянское,Отсутствует,кв.32,в.1,13.0,АнгоянскоеАнгоянскоеОтсутствуеткв.32в.1,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,500.0,13.0
72,Ангоянское,Ангоянское,Отсутствует,кв.32,в.1,13.0,АнгоянскоеАнгоянскоеОтсутствуеткв.32в.1,2022-06-29,NaN,2.3,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,400.0,13.0
2919,Бабушкинское,Бабушкинское,Отсутствует,кв.91,в.2,6.4,БабушкинскоеБабушкинскоеОтсутствуеткв.91в.2,NaT,NaN,0.6,Республика Бурятия,Бабушкинское,Бабушкинское,Отсутствует,91,2,7777.0,6.4


In [133]:
both_df.drop(columns=['1','2','3','4','5','6','7','8','9','Площадь лесотаксационного выдела, га'],axis=1,inplace=True)

In [134]:
both_df.head(1)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,ID,10,Площадь
69,Ангоянское,Ангоянское,Отсутствует,кв.32,в.1,АнгоянскоеАнгоянскоеОтсутствуеткв.32в.1,13.0,13.0


In [135]:
# # Очищаем от пробелов и строки названия урочища не заполнено
# both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
# both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.strip())

In [136]:
both_df.rename(columns={'Площадь':'Площадь из реестра УПП','10':'Старая площадь из таблицы 3'},inplace=True)

In [147]:
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Выделы с обновленной площадью'].append(r)

wb['Выделы с обновленной площадью'].column_dimensions['A'].width = 30
wb['Выделы с обновленной площадью'].column_dimensions['B'].width = 30
wb['Выделы с обновленной площадью'].column_dimensions['G'].width = 20
wb['Выделы с обновленной площадью'].column_dimensions['H'].width = 20


In [148]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
wb.save(f'{path_to_end_folder}/Данные для проверки корректности обновления площадей в таблице 3 от {current_time}.xlsx')

In [139]:
# Создаем новую таблицу 3 с обновленными данными
out_table3_df = merged_df[merged_df['_merge'] != 'left_only'] # отбираем общие значения и те что уже были в таблице 3

In [140]:
# Присваиваем колонке 10 значение колонки созданной колонки 10
out_table3_df['10'] = out_table3_df['Площадь']

In [141]:
out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь'],inplace=True)

In [142]:
out_table3_df.head(3)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
69,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2015.0
70,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2020.0
71,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2021.0


In [143]:
# Приводим даты к нормальному виду ДД.ММ.ГГГГ
out_table3_df['1'] = pd.to_datetime(out_table3_df['1'],errors='coerce',dayfirst=True)
out_table3_df['22'] = pd.to_datetime(out_table3_df['22'],errors='coerce',dayfirst=True)
out_table3_df['27'] = pd.to_datetime(out_table3_df['27'],errors='coerce',dayfirst=True)
out_table3_df['1'] = out_table3_df['1'].apply(create_doc_convert_date)
out_table3_df['22'] = out_table3_df['22'].apply(create_doc_convert_date)
out_table3_df['27'] = out_table3_df['27'].apply(create_doc_convert_date)

In [144]:
out_table3_df.head(3)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
69,29.06.2022,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2015.0
70,29.06.2022,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2020.0
71,29.06.2022,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,Отсутствует,32,1,13.0,...,0,0,,0.0,0.0,1.0,201.0,NaN,0.0,2021.0


In [145]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
itog = load_workbook(header_table3) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(out_table3_df,index=False,header=False):
    itog['Лист1'].append(row)

In [146]:
itog.save(f'{path_to_end_folder}/Таблица 3 с присвоенными площадями из реестра УПП от {current_time}.xlsx')